# TP 3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [2]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


### Récupération des données

In [3]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Programmation dynamique

In [4]:
function constructionC(price, weight, capacity)
    # Fonction pour construire la matrice C

    
    # Initialisation de la matrice
    c = zeros(length(price),capacity+1)
    i = 1

    while(i<= length(price))
        # Initialisation des colonnes à 2 car la première colonne reste à 0
        j = 2

        while(j<=capacity+1)
            # Définition des lignes et colonnes précédentes ou en enlevant le poids
            # pour éviter de sortir de la matrice
            i_prec = max(i-1,1)
            j_moins_poids = max(1,j-weight[i])
            # Calcul C récurrence pour comparaison
                # Cas sans ajout du poids
            c_prec = c[i_prec,j]
                # Cas ajout du poids à partir du cas où le poids max était plus faible
            c_actuel_ajout = (i==1 ? 0 : c[i_prec,j_moins_poids])+price[i]
                    # Test admissibilité, respect contrainte + cas particulier de la première ligne
            if (weight[i]>j-1)
                c_actuel_ajout = 0
            end
            
            # Comparaison des C obtenues (relation récurrence) pour maximiser le gain
            c[i,j] = max(c_prec,c_actuel_ajout)

            # Avancement colonne
            j+=1

        end
        # Avancement ligne
        i+=1

    end

    return c

end

function analyseC(c,price,weight,capacity)

    # Initialisation des résultats
    Bestsol, ObjSol, BestProfit = zeros(length(price)), [], -1
    # Début analyse dans le cas de poids correspondant à l'étude et de gain maximal
    i, j = length(price), capacity + 1
    # Récupération meilleur profit
    BestProfit = c[i,j]
    # Parcours descendant des lignes (i décroit)
    while (i > 1)
        
        # Si le gain est égal à celui de la ligne précédente, alors l'objet n'a pas été pris
        if c[i-1,j] == c[i,j]
            Bestsol[i] = 0.0
        # Sinon le gain à augmenter de la valeur ajoutée de l'objet
        else
            Bestsol[i] = 1.0
            push!(ObjSol,i)
            # On observe le gain pour un sac de capacité moindre et donc de poids occupé maximal sans l'objet validé
            j -= weight[i]
            # Vérification de non débordement de la matrice
            if j <= 0
                j = 1
            end
        end
        # Analyse ligne suivante : si l'objet précédent est présent ou non
        i-=1
    end
    # Analyse du premier objet potentiellement ajouté à part car pas possible d'observer les lignes précédentes.
    if c[i,j] != 0
        # Si le gain est non nul, alors ce premier objet est ajouté au sac
        Bestsol[i] = 1.0
        push!(ObjSol,i)
    end 

    return Bestsol, ObjSol, BestProfit
end

function SolveKnapInstanceDym(filename, affichage)

    price, weight, capacity = readKnaptxtInstance(filename)

    if affichage
        println("\nConstruction de la matrice C ...")
    end
    c = constructionC(price, weight, capacity)
    if affichage
        println("\n... Matrice C contruite")
    end
    #println("\nMatrice C obtenue ",c)
    if affichage
        println("\nAnalyse de la matrice C ...\n")
    end
    Bestsol,ObjSol,BestProfit = analyseC(c,price, weight, capacity)
    if affichage
        println("\n... Matrice C analysée")

        println("\n******\n\nOptimal value = ", BestProfit)
        if (length(Bestsol)<=100)
            println("\nOptimal (Matrice 0 objet pas présent, 1 objet présent) sol=", Bestsol)
        end
        println("\nOptima (objet présent) x=",ObjSol)
    end
    return BestProfit,Bestsol,ObjSol

end

SolveKnapInstanceDym (generic function with 1 method)

### Affichage du résultat final

In [6]:

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/test.opb.txt",true)

println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal (Matrice 0 objet pas présent, 1 objet présent) sol=", Bestsol, "\n\nOptima (objet présent) x=",ObjSol)



Construction de la matrice C ...

... Matrice C contruite

Analyse de la matrice C ...


... Matrice C analysée



******

Optimal value = 65.0

Optimal (Matrice 0 objet pas présent, 1 objet présent) sol=

[0.0, 1.0, 0.0, 1.0]

Optima (objet présent) x=

Any[4, 2]
  0.447293 seconds (539.81 k allocations: 29.897 MiB, 99.47% compilation time)

******

Optimal value = 65.0

Optimal (Matrice 0 objet pas présent, 1 objet présent) sol=[0.0, 1.0, 0.0, 1.0]

Optima (objet présent) x=Any[4, 2]


Par construction récurcive de la matrice C, pour chaque valeur $C_{i,j}$, on a la valeur maximale par ajout ou non des objets $k \in [1;...;i]$ respectant la capacité maximale $j$. Ainsi la valeur de la dernière ligne et de la dernière colonne correspond à la valeur maximale obtenue après ajout ou non de tout les poids dans le cas de la capacité maximale de l'étude de départ. En inversant la construction de la matrice, on peut retrouver les objets ajoutés dans le sac et ainsi avoir la meilleur solution.

### Programme Branch-and-Bound (TP2)

In [7]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end
function TestsSondabilite(listobjs, listvals, price, weight, capacity, borneinf, bornesup, BestProfit, Bestsol, affichage)
    TA, TO, TR = false, false, false
    
    current_weight = 0
    current_price = 0
    for i in 1:length(listobjs)
        if listvals[i] == 1
            current_weight += weight[listobjs[i]]
            current_price += price[listobjs[i]]
        end
               
    end

    if current_weight > capacity  # Test d'Admissibilité
        TA = true
        if affichage
            println("TA")
        end
    elseif (bornesup < BestProfit)  # Test d'optimalite
        TO = true
        if affichage
            println("TO")
        end
    elseif borneinf == bornesup # Test de resolution
        TR = true
        if affichage
            println("TR")
        end
        if bornesup >= BestProfit
            Bestsol = copy(listobjs.*listvals)
            BestProfit = bornesup
            if affichage
                println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
            end
        end
    else
        if affichage
            println("non sondable")
        end
    end
    return TA, TO, TR, Bestsol, BestProfit
end

function SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, n,  price, weight, affichage)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left  
    
    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj, maxr = 1, 0, -1.0
    while((i <= n))
        if (!(i in listobjs)) && ((price[i]/weight[i]) >= maxr)
            maxr = (price[i]/weight[i])
            obj = i
        end        
        i+=1
    end

    if affichage
        println("\nbranch on object ", obj, "\n")
    end
    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes, affichage)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            if affichage
                println("\nFINISHED")
            end
            stop=true
        end
    else
        #the root node was sondable
        if affichage
            println("\nFINISHED")
        end
        stop=true
    end
    return stop 
end
function Bornes_1!(price, weight, capacity, listobjs, listvals)
    
    placeDispo, prixEnCours, obj, maxr = capacity, 0.0, 0, -1.0
    for i in 1:length(listobjs)
        if listvals[i] == 1
            placeDispo -= weight[listobjs[i]]
            prixEnCours += price[listobjs[i]]
        end
    end 

    if (length(listobjs) < length(price))
        for i in 1:length(price)
            if (!(i in listobjs)) && ((price[i]/weight[i]) >= maxr)
                maxr = (price[i]/weight[i])
                obj = i
            end
        end
        facteurObj = placeDispo/weight[obj]

        borneinf, bornesup = prixEnCours, (prixEnCours+facteurObj*price[obj])
    else 
        borneinf, bornesup = prixEnCours, prixEnCours
    end
    return borneinf, bornesup

end

function Bornes_2!(price, weight, capacity, listobjs, listvals)
    placeDispo, prixEnCours, listeAjoutee, obj = capacity, 0.0, [], -1
    for i in 1:length(listobjs)
        if listvals[i] == 1
            placeDispo -= weight[listobjs[i]]
            prixEnCours += price[listobjs[i]]
        end
    end
    
    if (length(listobjs) < length(price)) && (placeDispo > 0)
        prixMax = prixEnCours
        while (placeDispo > 0) && ((length(listobjs) + length(listeAjoutee)) < length(price)) 
            maxr, obj = -1.0, 0
            for i in 1:length(price)
                if (!(i in listobjs)) && (!(i in listeAjoutee)) && ((price[i]/weight[i]) >= maxr)
                    maxr = (price[i]/weight[i])
                    obj = i
                end
            end
            push!(listeAjoutee,obj)

            placeDispo -= weight[obj]
            prixMax += price[obj]
        end
        
        if placeDispo < 0    
            placeDispo += weight[obj]
            prixMax -= price[obj]
        end
        
        facteurObj = placeDispo/weight[obj]  
        return prixEnCours,prixMax+price[obj]*facteurObj
    else
        return prixEnCours, prixEnCours
    end
end
function SolveKnapInstanceBranch(filename, borne, affichage)

    price, weight, capacity = readKnaptxtInstance(filename)

    #model2, x = CreationModele_LP(price, weight, capacity)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    BestProfit=-1
    Bestsol=[]
    Bestvals=[]

    current_node_number=0
    stop = false

    while(!stop)
        if affichage
            println("\nNode number ", current_node_number, ": \n---------------\n")
        end
        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        #=
        #create LP of current node
        MajModele_LP!(model2, x, listobjs, listvals)
        
        println(model2)
        
        print("Solve the LP model of the current node to compute its bound: start ... ")

        status = optimize!(model2)

        println("... end"); 

        print(": Solution LP")
        if(termination_status(model2) == MOI.INFEASIBLE)#(has_values(model2))
            print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
        else
            print(" ", objective_value(model2))
            [print("\t", name(v),"=",value(v)) for v in all_variables(model2)] 
        end
        println(" "); 

        =#

        #println("listobjs, listvals, listnodes", listobjs, listvals, listnodes)
        #Calcul des bornes
        if borne == "borne1"
            borneinf, bornesup = Bornes_1!(price, weight, capacity, listobjs, listvals)
        else borne == "borne2"
            borneinf, bornesup = Bornes_2!(price, weight, capacity, listobjs, listvals)
        end
        if (affichage==true)
            println("bornes :", [borneinf, bornesup])
            println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
        
        TA, TO, TR, Bestsol, BestProfit = TestsSondabilite(listobjs, listvals, price, weight, capacity, borneinf, bornesup, BestProfit, Bestsol, affichage)
        
        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        
        if(!is_node_sondable)
            SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, length(price), price, weight, affichage)
        else
            stop = ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes, affichage)        
        end
        
        
        
        #Reset_allLP!(model2, x)

        current_node_number = current_node_number + 1
    end
    if affichage
        println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
    end
    return BestProfit, filter(p->(p>0),Bestsol), trParentnodes, trChildnodes, trNamenodes, listvals

end

SolveKnapInstanceBranch (generic function with 1 method)

In [8]:
function contenu_equiv(array1,array2)
    #Fonction renvoyant true si les deux arrays contiennent les mêmes
    #valeurs (sans prendre en compte la position ni le nombre de fois)

    for elt1 in array1
        if (isempty(findall(x->x==elt1,array2)))
            return false
        end
    end

    for elt2 in array2
        if (isempty(findall(x->x==elt2,array1)))
            return false
        end
    end
    return true
end

contenu_equiv (generic function with 1 method)

In [9]:
println("\n******\n Cas dynamique\n")

@time BestProfit, Bestsol, ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)

println("\n******\n Cas Branch-and-Bound\n")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2, listvals = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)

println("\n******\n Comparaison :\n Même profit : ",BestProfit==BestProfit2,"\n Même solution : ",contenu_equiv(ObjSol,Bestsol2))


******
 Cas dynamique



  0.004678 seconds (102.72 k allocations: 1.725 MiB)

Optimal value = 999.0
Optimal x=Any[17, 15, 6, 3]

******
 Cas Branch-and-Bound

  0.203089 seconds (258.09 k allocations: 12.159 MiB, 96.98% compilation time)



Optimal value = 999.0
Optimal x=[15.0, 17.0, 3.0, 6.0]

******
 Comparaison :
 Même profit : 

true
 Même solution : true


In [10]:
println("\n******\n Cas dynamique")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2 = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_1_-1894.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)

println("\n******\n Comparaison :\n Même profit : ",BestProfit==BestProfit2,"\n Même solution : ",contenu_equiv(ObjSol,Bestsol2))


******
 Cas dynamique
  0.011508 seconds (263.20 k allocations: 4.413 MiB)

Optimal value = 1894.0
Optimal x=Any[47, 44, 30, 27, 25, 21, 20, 13, 2]

******
 Cas Branch-and-Bound


  0.036010 seconds (158.64 k allocations: 2.828 MiB)

Optimal value = 1894.0
Optimal x=[30.0, 21.0, 13.0, 47.0, 27.0, 2.0, 25.0, 44.0, 20.0]

******
 Comparaison :
 Même profit : true
 Même solution : true


In [11]:
println("\n******\n Cas dynamique")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/weakly_correlated/knapPI_2_100_1000_1_-1514.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2 = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/weakly_correlated/knapPI_2_100_1000_1_-1514.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)

println("\n******\n Comparaison :\n Même profit : ",BestProfit==BestProfit2,"\n Même solution : ",contenu_equiv(ObjSol,Bestsol2))


******
 Cas dynamique
  0.025479 seconds (530.10 k allocations: 8.867 MiB)

Optimal value = 1514.0
Optimal x=Any[85, 71, 57, 49, 45, 38, 33, 24, 11]

******
 Cas Branch-and-Bound


  0.155055 seconds (351.74 k allocations: 5.882 MiB)

Optimal value = 1514.0
Optimal x=[38.0, 24.0, 33.0, 71.0, 57.0, 85.0, 45.0, 11.0, 49.0]

******
 Comparaison :
 Même profit : true
 Même solution : true


On peut observer sur les différents tests ci-dessus que la programmation dynamique est plus rapide que la programmation par Branch and Bound, surtout pour des cas avec beaucoup de données, en obtenant des résultats équivalents. Cela est logique car la complexité pour la programmation dynamique est $o(nQ)$ (avec $n$ le nombre de donnée et $Q$ la capacité maximale du sac), alors que la complexité pour le Branch and Bound est $o(2^n)$.

In [12]:
println("\n******\n Cas dynamique")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/similar_weights/knapPI_9_100_1000_2_-997.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2 = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/similar_weights/knapPI_9_100_1000_2_-997.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)

println("\n******\n Comparaison :\n Même profit : ",BestProfit==BestProfit2,"\n Même solution : ",contenu_equiv(ObjSol,Bestsol2))


******
 Cas dynamique


 11.187907 seconds (296.08 M allocations: 4.707 GiB, 8.43% gc time)

Optimal value = 997.0
Optimal x=Any[119]

******
 Cas Branch-and-Bound


  3.295746 seconds (2.59 M allocations: 42.406 MiB)

Optimal value = 997.0
Optimal x=[119.0]

******
 Comparaison :
 Même profit : true
 Même solution : true


In [13]:
println("\n******\n Cas dynamique\n")

@time BestProfit, Bestsol, ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/circle/knapPI_16_20_1000_1_-2291.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)

println("\n******\n Cas Branch-and-Bound\n")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2, listvals = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/circle/knapPI_16_20_1000_1_-2291.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)

println("\n******\n Comparaison :\n Même profit : ",BestProfit==BestProfit2,"\n Même solution : ",contenu_equiv(ObjSol,Bestsol2))


******
 Cas dynamique

  0.004993 seconds (102.73 k allocations: 1.725 MiB)

Optimal value = 2291.0
Optimal x=Any[17, 15, 6, 3]

******
 Cas Branch-and-Bound

  0.003781 seconds (31.24 k allocations: 647.816 KiB)

Optimal value = 2291.0
Optimal x=[15.0, 6.0, 17.0, 3.0]

******
 Comparaison :
 Même profit : true
 Même solution : true


In [14]:
println("\n******\n Cas dynamique")

@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/multiple_strongly_correlated/knapPI_14_50_1000_1_-2586.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)
println("\n******\n Cas Branch-and-Bound")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2 = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/multiple_strongly_correlated/knapPI_14_50_1000_1_-2586.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)

println("\n******\n Comparaison :\n Même profit : ",BestProfit==BestProfit2,"\n Même solution : ",contenu_equiv(ObjSol,Bestsol2))


******
 Cas dynamique


  0.011689 seconds (260.90 k allocations: 4.379 MiB)

Optimal value = 2586.0
Optimal x=Any[46, 45, 43, 32, 30, 22, 15, 1]

******
 Cas Branch-and-Bound
  0.003048 seconds (14.97 k allocations: 299.806 KiB)

Optimal value = 2586.0
Optimal x=[15.0, 45.0, 43.0, 1.0, 32.0, 22.0, 30.0, 46.0]

******
 Comparaison :
 Même profit : true
 Même solution : true


Dans les tests ci-dessus, nous pouvons remarquer que le Branch and Bound est plus rapide.
Dans le premier test, le Branch and Bound trouve directement la solution où le sac ne prend qu'un objet, ce qui fait que le parcours de l'arbre est rapide, alors que la programmation dynamique va créer et parcourir son tableau de taille $nQ$. Dans le second test, le nombre de donnée étant suffisamment petit ($n$) pour que les complexités soit proches pour les deux programmes et ainsi, sachant que $o(2^n)$ est la complexité maximale du Branch and Bound, et donc il est possible que les données permettes de trouver la soltution rapidement, on a le Branch and Bound plus rapide que le dynamique. Cela se remarque aussi dans le troisème test où le Branch and Bound est plus rapide, ce qui est surement dû au fait que les données sont fortement corrélées.

In [15]:
println("\n******\n Cas dynamique\n")

@time BestProfit, Bestsol, ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/uncorrelated/knapPI_1_50_10000_3_-71860.opb.txt",false)

println("\nOptimal value = ", BestProfit, "\nOptimal x=", ObjSol)

println("\n******\n Cas Branch-and-Bound\n")

@time BestProfit2, Bestsol2, trParentnodes, trChildnodes, trNamenodes2, listvals = SolveKnapInstanceBranch("instancesETU/KNAPnewformat/uncorrelated/knapPI_1_50_10000_3_-71860.opb.txt", "borne2",false)

println("\nOptimal value = ", BestProfit2, "\nOptimal x=", Bestsol2)

println("\n******\n Comparaison :\n Même profit : ",BestProfit==BestProfit2,"\n Même solution : ",contenu_equiv(ObjSol,Bestsol2))


******
 Cas dynamique



  0.403447 seconds (10.70 M allocations: 174.601 MiB, 4.40% gc time)

Optimal value = 139416.0
Optimal x=Any[141, 140, 130, 128, 126, 121, 119, 89, 76, 69, 63, 60, 59, 33, 30, 26, 24, 11, 5, 1]

******
 Cas Branch-and-Bound

  0.039247 seconds (110.88 k allocations: 1.778 MiB)

Optimal value = 139416.0
Optimal x=[24.0, 128.0, 76.0, 33.0, 126.0, 89.0, 26.0, 121.0, 5.0, 130.0, 140.0, 119.0, 1.0, 141.0, 69.0, 59.0, 11.0, 63.0, 60.0, 30.0]

******
 Comparaison :
 Même profit : true
 Même solution : true


Ce test montre que les deux programmes peuvent obtenir un même profit optimal (139416) qui est mieux que celui attendu (indiqué dans le nom du test, ici 71860). Cela est intriguant et intéressant, car il peux indiquer que ces méthodes sont meilleurs que celle utilisé à la base lors de la création des données, ou soit nos programmes sont faux, mais cela semble peu probable car on obtient bien ce profit pour les objets choisis avec un poids total de 9770 pour une capacité maximale de 9868.

In [16]:
BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/strongly_correlated_span/knapPI_13_2000_1000_3_-33075.opb.txt",true)


Construction de la matrice C ...



... Matrice C contruite

Analyse de la matrice C ...


... Matrice C analysée

******

Optimal value = 49803.0

Optima (objet présent) x=Any[473, 472, 471, 469, 468, 467, 464, 462, 461, 459, 458, 457, 456, 453, 450, 449, 446, 444, 443, 441, 436, 435, 434, 433, 432, 431, 430, 429, 428, 425, 424, 420, 417, 414, 412, 408, 404, 402, 401, 400, 397, 394, 393, 392, 388, 384, 383, 381, 380, 379, 375, 374, 373, 371, 366, 365, 364, 361, 355, 354, 353, 352, 350, 349, 347, 345, 343, 342, 341, 340, 339, 338, 337, 336, 335, 334, 333, 331, 329, 326, 319, 318, 317, 316, 314, 313, 309, 304, 303, 301, 300, 299, 297, 292, 290, 289, 288, 286, 285, 284, 278, 277, 276, 275, 273, 271, 270, 269, 265, 264, 261, 260, 259, 258, 253, 252, 250, 249, 248, 242, 241, 239, 236, 235, 233, 232, 229, 227, 225, 222, 221, 219, 216, 215, 213, 208, 207, 204, 198, 197, 193, 191, 190, 189, 187, 186, 185, 184, 183, 182, 178, 177, 176, 175, 174, 172, 171, 168, 167, 165, 164, 163, 162, 159, 158, 155, 152, 147, 141, 137, 136, 135

(49803.0, [1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Any[473, 472, 471, 469, 468, 467, 464, 462, 461, 459  …  25, 17, 15, 12, 11, 10, 9, 6, 2, 1])

In [17]:
price, weight, capacity = readKnaptxtInstance("instancesETU/KNAPnewformat/strongly_correlated_span/knapPI_13_2000_1000_3_-33075.opb.txt")
#Objs = [473, 472, 471, 469, 468, 467, 464, 462, 461, 459, 458, 457, 456, 453, 450, 449, 446, 444, 443, 441, 436, 435, 434, 433, 432, 431, 430, 429, 428, 425, 424, 420, 417, 414, 412, 408, 404, 402, 401, 400, 397, 394, 393, 392, 388, 384, 383, 381, 380, 379, 375, 374, 373, 371, 366, 365, 364, 361, 355, 354, 353, 352, 350, 349, 347, 345, 343, 342, 341, 340, 339, 338, 337, 336, 335, 334, 333, 331, 329, 326, 319, 318, 317, 316, 314, 313, 309, 304, 303, 301, 300, 299, 297, 292, 290, 289, 288, 286, 285, 284, 278, 277, 276,275, 273, 271, 270, 269, 265, 264, 261, 260, 259, 258, 253, 252, 250, 249, 248, 242, 241, 239, 236, 235, 233, 232, 229, 227, 225, 222, 221, 219, 216, 215, 213, 208, 207, 204, 198, 197, 193, 191, 190, 189, 187, 186, 185, 184, 183, 182, 178, 177, 176, 175, 174, 172, 171, 168, 167, 165, 164, 163, 162, 159, 158, 155, 152, 147, 141, 137, 136, 135, 133, 132, 127, 121, 118, 117, 114, 113, 112, 108, 107, 103, 100, 99, 97, 96, 94, 91, 89, 88, 86, 85, 84, 83, 79, 76, 74, 73, 72, 71, 68, 67, 66, 65, 62, 61, 60, 54, 52, 50, 45, 43, 40, 39, 38, 36, 35, 34, 33, 32, 25, 17, 15, 12, 11, 10, 9, 6, 2, 1]
Objs = ObjSol
println("\nProfit avec les objets sélectionnés : ",sum(price[Objs]),"\nPoids total : ",sum(weight[Objs]),"\nCapacité : ",capacity)


Profit avec les objets sélectionnés : 49803
Poids total : 24263
Capacité : 24277


Comme précédemment, on a un autre jeu de donnée telle que le profit maximal obtenu est meilleur que celui attendu, $49803 > 33075$, tout en respectant la capacité : $24263\leq24277$. 

In [18]:
BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_2000_1000_5_-93189.opb.txt",true)


Construction de la matrice C ...



... Matrice C contruite

Analyse de la matrice C ...


... Matrice C analysée

******

Optimal value = 149089.0

Optima (objet présent) x=Any[9991, 9976, 9973, 9944, 9938, 9932, 9927, 9917, 9898, 9890, 9858, 9847, 9835, 9831, 9818, 9811, 9808, 9781, 9767, 9745, 9744, 9740, 9734, 9721, 9714, 9712, 9705, 9687, 9669, 9640, 9607, 9606, 9603, 9587, 9579, 9575, 9548, 9530, 9518, 9514, 9506, 9505, 9503, 9479, 9477, 9469, 9399, 9392, 9391, 9388, 9370, 9368, 9310, 9308, 9296, 9265, 9261, 9257, 9251, 9243, 9228, 9215, 9213, 9171, 9170, 9136, 9130, 9128, 9119, 9112, 9111, 9083, 9075, 9058, 9039, 9036, 9032, 9029, 9021, 9002, 9000, 8995, 8947, 8941, 8928, 8923, 8890, 8878, 8856, 8851, 8846, 8828, 8787, 8785, 8783, 8768, 8762, 8759, 8756, 8754, 8732, 8728, 8718, 8716, 8715, 8714, 8706, 8703, 8695, 8681, 8663, 8655, 8644, 8637, 8635, 8597, 8585, 8580, 8574, 8556, 8555, 8550, 8543, 8530, 8522, 8520, 8517, 8479, 8472, 8461, 8438, 8421, 8417, 8410, 8365, 8329, 8321, 8315, 8312, 8288, 8287, 8286, 8281,

(149089.0, [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Any[9991, 9976, 9973, 9944, 9938, 9932, 9927, 9917, 9898, 9890  …  86, 77, 75, 65, 47, 30, 27, 21, 13, 2])

In [19]:
price, weight, capacity = readKnaptxtInstance("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_2000_1000_5_-93189.opb.txt")
#Objs = [9991, 9976, 9973, 9944, 9938, 9932, 9927, 9917, 9898, 9890, 9858, 9847, 9835, 9831, 9818, 9811, 9808, 9781, 9767, 9745, 9744, 9740, 9734, 9721, 9714, 9712, 9705, 9687, 9669, 9640, 9607, 9606, 9603, 9587, 9579, 9575, 9548, 9530, 9518, 9514, 9506, 9505, 9503, 9479, 9477, 9469, 9399, 9392, 9391, 9388, 9370, 9368, 9310, 9308, 9296, 9265, 9261, 9257, 9251, 9243, 9228, 9215, 9213, 9171, 9170, 9136, 9130, 9128, 9119, 9112, 9111, 9083, 9075, 9058, 9039, 9036, 9032, 9029, 9021, 9002, 9000, 8995, 8947, 8941, 8928, 8923, 8890, 8878, 8856, 8851, 8846, 8828, 8787, 8785, 8783, 8768, 8762, 8759, 8756, 8754, 8732, 8728, 8718, 8716, 8715, 8714, 8706, 8703, 8695, 8681, 8663, 8655, 8644, 8637, 8635, 8597, 8585, 8580, 8574, 8556, 8555, 8550, 8543, 8530, 8522, 8520, 8517, 8479, 8472, 8461, 8438, 8421, 8417, 8410, 8365, 8329, 8321, 8315, 8312, 8288, 8287, 8286, 8281, 8277, 8270, 8263, 8250, 8242, 8237, 8228, 8207, 8205, 8182, 8168, 8148, 8128, 8117, 8096, 8088, 8085, 8041, 8040, 8029, 8024, 8022, 8020, 7983, 7968, 7952, 7949, 7947, 7932, 7927, 7925, 7905, 7891, 7889, 7886, 7885, 7882, 7879, 7869, 7866, 7859, 7858, 7842, 7838, 7836, 7824, 7823, 7819, 7815, 7814, 7807, 7805, 7795, 7790, 7785, 7761, 7733, 7717, 7698, 7695, 7688, 7685, 7673, 7665, 7662, 7645, 7642, 7638, 7634, 7626, 7623, 7573, 7567, 7557, 7543, 7538, 7531, 7527, 7515, 7514, 7497, 7493, 7445, 7423, 7404, 7398, 7397, 7395, 7392, 7391, 7384, 7367, 7364, 7351, 7324, 7318, 7317, 7313, 7307, 7301, 7298, 7294, 7272, 7269, 7268, 7261, 7253, 7241, 7236, 7228, 7226, 7221, 7220, 7218, 7217, 7211, 7204, 7190, 7188, 7165, 7164, 7145, 7144, 7140, 7138, 7099, 7091, 7087, 7078, 7076, 7073, 7071, 7062, 7059, 7057, 7044, 7033, 7029, 7028, 7014, 7010, 7007, 6995, 6994, 6989, 6968, 6967, 6964, 6960, 6955, 6944, 6937, 6926, 6900, 6897, 6885, 6879, 6877, 6875, 6873, 6861, 6842, 6807, 6805, 6801, 6793, 6790, 6779, 6761, 6733, 6726, 6719, 6714, 6699, 6695, 6675, 6650, 6640, 6623, 6604, 6599, 6586, 6585, 6576, 6572, 6547, 6543, 6534, 6533, 6531, 6524, 6521, 6492, 6491, 6488, 6480, 6478, 6477, 6469, 6458, 6456, 6438, 6435, 6408, 6387, 6376, 6358, 6352, 6351, 6347, 6341, 6327, 6325, 6298, 6285, 6281, 6279, 6268, 6259, 6243, 6241, 6240, 6239, 6237, 6234, 6232, 6215, 6212, 6183, 6166, 6163, 6158, 6142, 6122, 6107, 6104, 6102, 6086, 6069, 6060, 6058, 6052, 6036, 6023, 6019, 6017, 6008, 6007, 5987, 5986, 5984, 5982, 5962, 5960, 5940, 5937, 5933, 5926, 5866, 5862, 5861, 5858, 5856, 5835, 5824, 5819, 5805, 5803, 5802, 5796, 5794, 5779, 5776, 5746, 5743, 5738, 5737, 5725, 5721, 5717, 5704, 5660, 5657, 5646, 5637, 5623, 5611, 5606, 5602, 5575, 5571, 5537, 5488, 5483, 5466, 5460, 5449, 5438, 5434, 5429, 5426, 5415, 5413, 5403, 5401, 5393, 5374, 5370, 5363, 5361, 5338, 5325, 5322, 5313, 5308, 5288, 5287, 5257, 5234, 5224, 5215, 5199, 5198, 5180, 5175, 5168, 5164, 5161, 5149, 5147, 5146, 5134, 5132, 5104, 5082, 5066, 5052, 5047, 5042, 5036, 5024, 5014, 5013, 5003, 4986, 4985, 4974, 4965, 4959, 4956, 4937, 4935, 4929, 4920, 4899, 4897, 4872, 4829, 4824, 4794, 4789, 4784, 4777, 4776, 4750, 4748, 4738, 4720, 4702, 4701, 4699, 4697, 4696, 4695, 4693, 4680, 4678, 4654, 4644, 4623, 4620, 4611, 4609, 4601, 4600, 4593, 4586, 4585, 4577, 4561, 4557, 4552, 4542, 4536, 4533, 4532, 4519, 4502, 4479, 4466, 4457, 4438, 4437, 4434, 4407, 4400, 4398, 4397, 4390, 4388, 4381, 4375, 4373, 4370, 4367, 4366, 4364, 4363, 4359, 4355, 4346, 4341, 4335, 4329, 4320, 4310, 4301, 4298, 4290, 4262, 4261, 4247, 4246, 4230, 4208, 4205, 4204, 4198, 4190, 4185, 4183, 4177, 4158, 4147, 4146, 4140, 4136, 4128, 4109, 4092, 4087, 4084, 4075, 4056, 4031, 4024, 4021, 4014, 4005, 3989, 3988, 3984, 3978, 3968, 3952, 3949, 3944, 3921, 3911, 3908, 3901, 3876, 3872, 3849, 3847, 3845, 3840, 3837, 3818, 3799, 3782, 3778, 3769, 3747, 3745, 3744, 3740, 3733, 3726, 3710, 3683, 3671, 3664, 3661, 3591, 3578, 3571, 3557, 3538, 3533, 3524, 3520, 3515, 3509, 3502, 3473, 3472, 3444, 3443, 3441, 3429, 3424, 3405, 3395, 3387, 3384, 3381, 3332, 3327, 3309, 3300, 3289, 3278, 3259, 3251, 3247, 3245, 3214, 3200, 3199, 3168, 3167, 3144, 3142, 3129, 3125, 3114, 3107, 3086, 3075, 3074, 3060, 3054, 3043, 3040, 3031, 3023, 2955, 2944, 2934, 2929, 2913, 2909, 2896, 2894, 2892, 2883, 2877, 2868, 2860, 2844, 2842, 2829, 2826, 2818, 2814, 2797, 2790, 2783, 2769, 2762, 2754, 2743, 2735, 2713, 2709, 2702, 2699, 2695, 2694, 2691, 2680, 2675, 2673, 2658, 2643, 2641, 2639, 2637, 2616, 2602, 2585, 2580, 2571, 2568, 2560, 2550, 2532, 2527, 2523, 2508, 2501, 2478, 2476, 2475, 2474, 2464, 2453, 2452, 2448, 2419, 2406, 2391, 2379, 2372, 2368, 2366, 2361, 2317, 2295, 2287, 2267, 2259, 2253, 2233, 2218, 2217, 2214, 2194, 2191, 2184, 2171, 2158, 2154, 2123, 2119, 2115, 2096, 2091, 2090, 2079, 2053, 2021, 2019, 2014, 1994, 1985, 1979, 1969, 1946, 1943, 1900, 1891, 1878, 1875, 1854, 1850, 1836, 1835, 1775, 1760, 1734, 1732, 1711, 1699, 1696, 1691, 1690, 1682, 1680, 1677, 1661, 1658, 1657, 1649, 1645, 1638, 1634, 1624, 1618, 1616, 1615, 1606, 1600, 1573, 1568, 1562, 1556, 1551, 1541, 1492, 1487, 1475, 1473, 1472, 1468, 1465, 1464, 1452, 1446, 1440, 1437, 1417, 1412, 1407, 1386, 1380, 1377, 1370, 1354, 1352, 1346, 1345, 1339, 1315, 1310, 1303, 1287, 1274, 1268, 1262, 1225, 1221, 1207, 1206, 1204, 1199, 1197, 1194, 1190, 1188, 1187, 1171, 1168, 1160, 1150, 1147, 1106, 1101, 1098, 1079, 1008, 1002, 989, 987, 960, 959, 953, 947, 939, 927, 922, 910, 896, 893, 884, 856, 853, 852, 850, 843, 839, 834, 768, 765, 759, 758, 740, 725, 722, 699, 695, 676, 669, 666, 664, 663, 660, 638, 608, 607, 606, 598, 590, 585, 584, 575, 568, 563, 547, 539, 525, 506, 499, 491, 484, 480, 476, 473, 433, 424, 423, 392, 385, 376, 344, 324, 308, 303, 295, 293, 275, 274, 272, 269, 266, 243, 234, 212, 205, 204, 170, 165, 164, 158, 148, 121, 114, 107, 97, 90, 86, 77, 75, 65, 47, 30, 27, 21, 13, 2]
Objs = ObjSol
println("\nProfit avec les objets sélectionnés : ",sum(price[Objs]),"\nPoids total : ",sum(weight[Objs]),"\nCapacité : ",capacity)


Profit avec les objets sélectionnés : 149089
Poids total : 50689
Capacité : 50689


Autre cas de solution obtenu meilleure que celle attendue : $149089 > 93189$.


@time BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_1000_1000_1_-14390.opb.txt",true)

Vu que le temps de réalisation est moindre pour de grande base de donnée, on peut réaliser avec la programmation dynamique des tests que nous n'avions pas pu faire lors du TP2 comme avec la base de données ci-dessus.

In [21]:
BestProfit, Bestsol,ObjSol = SolveKnapInstanceDym("instancesETU/KNAPnewformat/uncorrelated/knapPI_1_10000_10000_2_-8334617.opb.txt",true)


Construction de la matrice C ...


OutOfMemoryError: OutOfMemoryError()

Cependant, la programmation dynamique est quand même limitée : la matrice C peut facilement être grande et donc être facilement à cours de mémoire, comme le cas précédent sur notre PC, ou bien avoir un temps trop long d'exécution.

### BONUS : Algorithme de Bellman-Ford

In [66]:
sommets = ["A","B","C","D","E","F"]
#arcsOrigines = ["A","A","B","C","E","E","D"]
arcsOrigines = [1,1,2,3,5,5,4]
#arcsTerminaisons = ["B","E","C","D","B","D","F"]
arcsTerminaisons = [2,5,3,4,2,4,6]
# arcs i : (sommets[arcsOrigines[i]],sommets[arcsTerminaisons[i]]) 
c = [0 3 0 0 5 0; 0 0 4 0 0 0 ; 0 0 0 2 0 0 ; 0 0 0 0 0 3 ; 0 -1 0 9 0 0 ; 0 0 0 0 0 0 ]
fkval = zeros(1,length(sommets))
fkmoinsval = []
for i in 2:length(sommets)
    fkval[i] = Inf
end
fkdep = []
fdepindice = ones(Int,1,length(sommets))
for i in 1:length(sommets)
    push!(fkdep,"")
end
fval = []
fdep = []

push!(fval,copy(fkval))
push!(fdep,copy(fkdep))
k = 1
while ((fkmoinsval != fkval) || (k >= length(sommets) + 1))
    fkmoinsval = copy(fkval)
    for i in 2:length(fkval)
        arcsTerminantsPari = findall(x->x==i,arcsTerminaisons)
        predDei = arcsOrigines[arcsTerminantsPari]
        
        fjkpred,jpred = findmin(fkmoinsval[predDei]+c[predDei,i])
        
        if (fjkpred <= fkval[i] && fjkpred != fkval[i])
            fkval[i] = fjkpred
            fkdep[i] = sommets[predDei[jpred]]
            fdepindice[i] = predDei[jpred]
        end
    end
    push!(fval,copy(fkval))
    push!(fdep,copy(fkdep))
    k +=1
end
if (k>= length(sommets) + 1)
    println("Présence cycle de longueur négative")
else
    println("\n Valeurs à chaque itération ",fval,"\nOrigines permettant cette valeur à chaque itération: ",fdep)

    chemin = []
    indice = length(sommets)
    push!(chemin,sommets[indice])
    while indice != 1
        push!(chemin,sommets[fdepindice[indice]])
        indice = fdepindice[indice]
    end
    chemin = reverse(chemin)
    println("\n Chemin : ",chemin)
end




 Valeurs à chaque itération Any[[0.0 Inf Inf Inf Inf Inf], [0.0 3.0 Inf Inf 5.0 Inf], [0.0 3.0 7.0 14.0 5.0 Inf], [0.0 3.0 7.0 9.0 5.0 17.0], [0.0 3.0 7.0 9.0 5.0 12.0], [0.0 3.0 7.0 9.0 5.0 12.0]]
Origines permettant cette valeur à chaque itération: Any[Any["", "", "", "", "", ""], Any["", "A", "", "", "A", ""], Any["", "A", "B", "E", "A", ""], Any["", "A", "B", "C", "A", "D"], Any["", "A", "B", "C", "A", "D"], Any["", "A", "B", "C", "A", "D"]]

 Chemin : Any["A", "B", "C", "D", "F"]
